> `hover` supports bulk-labeling audios through their URLs (which can be local).
>
> :bulb: Let's do a quickstart for audios and note what's different from texts.

-   <details open><summary>This page assumes that you have know the basics</summary>
    i.e. simple usage of `dataset` and `annotator`. Please visit the [quickstart tutorial](/hover/pages/tutorial/t0-quickstart) if you haven't done so.

</details>

## **Dataset for audios**

`hover` handles audios through their URL addresses. URLs are strings which can be easily stored, hashed, and looked up against. They are also convenient for rendering tooltips in the annotation interface.

Similarly to `SupervisableTextDataset`, we can build one for audios:

In [1]:
from hover.core.dataset import SupervisableAudioDataset
import pandas as pd

# this is a table of audio-MNIST (pronounced digit 0-9) urls, 100 audios per digit
example_csv_path = "https://raw.githubusercontent.com/phurwicz/hover-gallery/main/0.7.0/audio_mnist.csv"
df = pd.read_csv(example_csv_path).sample(frac=1).reset_index(drop=True)
df["SUBSET"] = "raw"
df.loc[500:800, 'SUBSET'] = 'train'
df.loc[800:900, 'SUBSET'] = 'dev'
df.loc[900:, 'SUBSET'] = 'test'

dataset = SupervisableAudioDataset.from_pandas(df, feature_key="audio", label_key="label")

# each subset can be accessed as its own DataFrame
dataset.dfs["raw"]().head(5)

🔵 SupervisableAudioDataset: Initializing...

🔵 SupervisableAudioDataset: Deduplicating...

🔵 SupervisableAudioDataset: --subset raw rows: 500 -> 500.

🔵 SupervisableAudioDataset: --subset train rows: 300 -> 300.

🔵 SupervisableAudioDataset: --subset dev rows: 100 -> 100.

...

🟢 SupervisableAudioDataset: Set up label encoder/decoder with 10 classes.

🟢 SupervisableAudioDataset: Population updater: latest population with 10 classes.

🔵 SupervisableAudioDataset: finished setting up bokeh elements.

🟢 SupervisableAudioDataset: finished initialization.

,audio,label,SUBSET
0,https://raw.githubusercontent.com/phurwicz/Aud...,ABSTAIN,raw
1,https://raw.githubusercontent.com/phurwicz/Aud...,ABSTAIN,raw
2,https://raw.githubusercontent.com/phurwicz/Aud...,ABSTAIN,raw
3,https://raw.githubusercontent.com/phurwicz/Aud...,ABSTAIN,raw
4,https://raw.githubusercontent.com/phurwicz/Aud...,ABSTAIN,raw


## **Vectorizer for audios**

We can follow a `URL -> content -> audio array -> vector` path.

In [2]:
import requests
from functools import lru_cache

@lru_cache(maxsize=10000)
def url_to_content(url):
    """
    Turn a URL to response content.
    """
    response = requests.get(url)
    return response.content

In [3]:
import librosa
from io import BytesIO

@lru_cache(maxsize=10000)
def url_to_audio(url):
    """
    Turn a URL to audio data.
    """
    data, sampling_rate = librosa.load(BytesIO(url_to_content(url)))
    return data, sampling_rate

-   <details open><summary>Caching and reading from disk</summary>
    This guide uses [`@wrappy.memoize`](https://erniethornhill.github.io/wrappy/) in place of `@functools.lru_cache` for caching.

    -   The benefit is that `wrappy.memoize` can persist the cache to disk, speeding up code across sessions.

    Cached values for this guide have been pre-computed, making it much master to run the guide.

</details>

In [4]:
import wrappy

@wrappy.memoize(cache_limit=10000, persist_path='custom_cache/audio_url_to_vector.pkl')
def vectorizer(url):
    """
    Averaged MFCC over time.
    Resembles word-embedding-average-as-doc-embedding for texts.
    """
    y, sr = url_to_audio(url)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=32)
    return mfcc.mean(axis=1)

ℹ Persisting __main__.vectorizer() output to
custom_cache/audio_url_to_vector.pkl.


## **Embedding and Plot**

This is exactly the same as in the quickstart, just switching to audio data:

In [5]:
# any kwargs will be passed onto the corresponding reduction
# for umap: https://umap-learn.readthedocs.io/en/latest/parameters.html
# for ivis: https://bering-ivis.readthedocs.io/en/latest/api.html
reducer = dataset.compute_nd_embedding(vectorizer, "umap", dimension=2)

Vectorizing:   0%|          | 0/1000 [00:00<?, ?it/s]

Vectorizing:   0%|          | 1/1000 [00:09<2:32:06,  9.14s/it]

Vectorizing:   0%|          | 2/1000 [00:09<1:04:17,  3.87s/it]

Vectorizing:   0%|          | 3/1000 [00:09<36:06,  2.17s/it]  

Vectorizing:   0%|          | 4/1000 [00:09<22:59,  1.38s/it]

...

Vectorizing: 100%|██████████| 1000/1000 [02:53<00:00,  4.99it/s]

Vectorizing: 100%|██████████| 1000/1000 [02:53<00:00,  5.78it/s]

🔵 SupervisableAudioDataset: Fit-transforming UMAP on 900 samples...

🔵 SupervisableAudioDataset: Transforming UMAP on 100 samples...

🟢 SupervisableAudioDataset: Computed 2-d embedding in columns ['embed_2d_0', 'embed_2d_1']

In [6]:
from hover.recipes.stable import simple_annotator

interactive_plot = simple_annotator(dataset)

# ---------- NOTEBOOK MODE: for your actual Jupyter environment ---------
# this code will render the entire plot in Jupyter
# from bokeh.io import show, output_notebook
# output_notebook()
# show(interactive_plot, notebook_url='https://localhost:8888')

-   <details open><summary>What's special for audios?</summary>
    **Tooltips**

    For text, the tooltip shows the original value.

    For audios, the tooltip embeds the audio based on URL.

    -   audios in the local file system shall be served through [`python -m http.server`](https://docs.python.org/3/library/http.server.html).
    -   they can then be accessed through `https://localhost:<port>/relative/path/to/file`.

    **Search**

    For text, the search widget is based on regular expressions.

    For audios, the search widget is based on vector cosine similarity.

    -   the `dataset` has remembered the `vectorizer` under the hood and passed it to the `annotator`.
    -   {== please [**let us know**](https://github.com/phurwicz/hover/issues/new) if you think there's a better way to search audios in this case. ==}
        -   dynamic time warping, due to its running time (> 10ms per pair for small 100x10 MFCC arrays), is too slow for search.
            -   we are experimenting with subsampled signals and pre-selected data points (by vector similarity, for example).

</details>